In [108]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


# PREPROCESSING

## 1- Load Dataset

In [109]:
data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv' , index_col='PassengerId')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Cabin         8494 non-null   object 
 3   Destination   8511 non-null   object 
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   object 
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
 11  Name          8493 non-null   object 
 12  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(6)
memory usage: 891.4+ KB


In [110]:
X = data.drop(['Transported']  , axis=1)
y = data['Transported']
X.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
PassengerId,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


### Seprate mixed column

In [111]:
X[['Cabin_deck' , 'Cabin_num' , 'Cabin_side']] = X['Cabin'].str.split('/', expand=True)
X = X.drop(['Cabin' , 'Name'] , axis=1)

## 2- Handeling missing values

In [112]:
X[X.isnull().any(axis=1)]

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_deck,Cabin_num,Cabin_side
PassengerId,,,,,,,,,,,,,
0006_02,Earth,True,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,G,0,S
0008_02,Europa,True,TRAPPIST-1e,34.0,False,0.0,0.0,NaN,0.0,0.0,B,1,P
0012_01,Earth,False,TRAPPIST-1e,31.0,False,32.0,0.0,876.0,0.0,0.0,NaN,NaN,NaN
0014_01,Mars,False,55 Cancri e,27.0,False,1286.0,122.0,NaN,0.0,0.0,F,3,P
0020_03,Earth,True,55 Cancri e,29.0,False,0.0,0.0,NaN,0.0,0.0,E,0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9250_01,Europa,False,TRAPPIST-1e,29.0,False,0.0,2972.0,NaN,28.0,188.0,E,597,P
9257_01,NaN,False,TRAPPIST-1e,13.0,False,39.0,0.0,1085.0,24.0,0.0,F,1892,P
9259_01,Earth,NaN,TRAPPIST-1e,44.0,False,1030.0,1015.0,0.0,11.0,NaN,F,1893,P


In [113]:
X.isnull().sum() #Check null rows

HomePlanet      201
CryoSleep       217
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Cabin_deck      199
Cabin_num       199
Cabin_side      199
dtype: int64

### Seprate categorical and numerical columns

In [114]:
categorical_features = [feature for feature in X.columns if len(X[feature].unique()) < 25]
numerical_features = [feature for feature in X.columns if feature not in categorical_features]
print("Categorical features:" , categorical_features)
print("Numerical features:" , numerical_features)

Categorical features: ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Cabin_deck', 'Cabin_side']
Numerical features: ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin_num']


### Handling the missing values for **numerical** columns

In [115]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[numerical_features])
X[numerical_features] = imputer.transform(X[numerical_features])
X[numerical_features].isnull().sum()

Age             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Cabin_num       0
dtype: int64

### handling the missing values for **categorical** columns

In [116]:
pd.set_option('future.no_silent_downcasting', True)
X[categorical_features] = X[categorical_features].fillna(X[categorical_features].mode().iloc[0])

In [117]:
X[categorical_features].isnull().sum()

HomePlanet     0
CryoSleep      0
Destination    0
VIP            0
Cabin_deck     0
Cabin_side     0
dtype: int64

## 3- Encoding the categorical data

In [118]:
X.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_deck,Cabin_num,Cabin_side
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,B,0.0,P
0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,F,0.0,S
0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,A,0.0,S
0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,A,0.0,S
0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,F,1.0,S


In [119]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, drop=None)
ct = ColumnTransformer(transformers=[('encoder',ohe , categorical_features)], remainder='passthrough')
X_transformed_df = ct.fit_transform(X)
cat_feature = ct.named_transformers_['encoder'].get_feature_names_out(categorical_features)

all_feature_names = list(cat_feature) + numerical_features
X_transformed_df =pd.DataFrame(X_transformed_df, columns=all_feature_names , index=X.index)

In [120]:
X_transformed_df

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,...,Cabin_deck_T,Cabin_side_P,Cabin_side_S,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_num
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0
0002_01,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,24.0,109.0,9.0,25.0,549.0,44.0,0.0
0003_01,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,58.0,43.0,3576.0,0.0,6715.0,49.0,0.0
0003_02,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,33.0,0.0,1283.0,371.0,3329.0,193.0,0.0
0004_01,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,16.0,303.0,70.0,151.0,565.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,41.0,0.0,6819.0,0.0,1643.0,74.0,98.0
9278_01,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,1499.0
9279_01,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,26.0,0.0,0.0,1872.0,1.0,0.0,1500.0


## 4- Encoding the label

In [121]:
y.head()

PassengerId
0001_01    False
0002_01     True
0003_01    False
0003_02    False
0004_01     True
Name: Transported, dtype: bool

In [122]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_transformed_df = le.fit_transform(y)
y_transformed_df = pd.Series(y_transformed_df , name=y.name , index=y.index)
y_transformed_df

PassengerId
0001_01    0
0002_01    1
0003_01    0
0003_02    0
0004_01    1
          ..
9276_01    0
9278_01    0
9279_01    1
9280_01    0
9280_02    1
Name: Transported, Length: 8693, dtype: int64

In [123]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed_df, y_transformed_df, test_size = 0.2, random_state = 1)

## 5- Feature scaling

In [124]:
from sklearn.preprocessing import StandardScaler
sta = StandardScaler()
X_train[numerical_features] = sta.fit_transform(X_train[numerical_features])
X_test[numerical_features] = sta.transform(X_test[numerical_features])

# Model Selection

In [125]:
import seaborn as sns
import matplotlib.pyplot as plt
from timeit import timeit

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.compose import make_column_selector as selector
from sklearn.pipeline import Pipeline

In [126]:
random_state = 42
models = dict()
models['Logistic Regression'] = LogisticRegression(random_state=random_state, max_iter=1000, class_weight='balanced')
models['Decision Tree'] = DecisionTreeClassifier(random_state=random_state, class_weight='balanced')
models['Random Forest'] = RandomForestClassifier(random_state=random_state, class_weight='balanced')
models['Extra Trees'] = ExtraTreesClassifier(random_state=random_state, class_weight='balanced')
models['Gradient Boosting'] = GradientBoostingClassifier(random_state=random_state)
models['Hist Gradient Boosting'] = HistGradientBoostingClassifier(random_state=random_state)
models['AdaBoost'] = AdaBoostClassifier(random_state=random_state)
models['SGD'] = SGDClassifier(random_state=random_state, class_weight='balanced')
models['SVC'] = SVC(class_weight='balanced', random_state=random_state)
models['Nearest Neighbor'] = KNeighborsClassifier(3)
models['Perceptron'] = Perceptron(random_state=random_state)

### Model evaluation function

In [127]:
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=5, 
                                 n_repeats=10, 
                                 random_state=1)
    scores = cross_val_score(model, X, y, 
                             scoring='roc_auc', 
                             cv=cv, n_jobs=-1)
    return scores

In [128]:
results , names = list() , list()
for name , model  in models.items():
    %time scores = evaluate_model(model, X_train, y_train)
    results.append(scores)
    names.append(name)
    print('* %s Score = %.3f StdDev = (%.3f)' % (name, np.mean(scores), np.std(scores)), '\n')

plt.figure(figsize=(10,8))
plt.boxplot(results, labels=names, showmeans=True)
plt.xticks(rotation=45)


In [129]:
final_model = HistGradientBoostingClassifier(random_state=random_state)
final_model.fit(X_train , y_train)

HistGradientBoostingClassifier(random_state=42)

In [130]:
from sklearn.metrics import accuracy_score

y_pred = final_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.8159861989649224


In [138]:
numerical_features

['Age',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Cabin_num']

In [139]:
X_test[numerical_features]

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_num
PassengerId,,,,,,,
0013_01,27.0,0.0,0.0,0.0,0.0,0.0,3
0018_01,19.0,0.0,9.0,0.0,2823.0,0.0,4
0019_01,31.0,0.0,0.0,0.0,0.0,0.0,0
0021_01,38.0,0.0,6652.0,0.0,181.0,585.0,1
0023_01,20.0,10.0,0.0,635.0,0.0,0.0,5
...,...,...,...,...,...,...,...
9266_02,34.0,0.0,0.0,0.0,0.0,0.0,1496
9269_01,42.0,0.0,847.0,17.0,10.0,144.0,NaN
9271_01,NaN,0.0,0.0,0.0,0.0,0.0,296


In [146]:
data_test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv', index_col='PassengerId')
X_test = data_test.copy()
X_test[['Cabin_deck' , 'Cabin_num' , 'Cabin_side']] = X_test['Cabin'].str.split('/', expand=True)
X_test = X_test.drop(['Cabin' , 'Name'] , axis=1)
X_test[numerical_features]
X_test[numerical_features] = imputer.transform(X_test[numerical_features])
X_test[numerical_features] = sta.transform(X_test[numerical_features])

X_test[categorical_features] = X_test[categorical_features].fillna(X_test[categorical_features].mode().iloc[0])
X_test = ct.transform(X_test)

In [160]:
y_pred = final_model.predict(X_test)
result = pd.DataFrame(y_pred==1 , index = data_test.index , columns=['Transported'])

In [162]:
result.to_csv("/kaggle/working/submission.csv")